In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings("ignore")

In [56]:
DATA_DIR = 'data/'
SUBSAMPLE_FEATURE_PREPROC = 0.01
SUBSAMPLE_TRAIN = 0.9
METER = 3

In [57]:
!ls {DATA_DIR}

building_metadata.csv  test.csv   weather_test.csv
sample_submission.csv  train.csv  weather_train.csv


# Load data 

In [58]:
building_metadata = pd.read_csv(DATA_DIR + 'building_metadata.csv')
weather_train = pd.read_csv(DATA_DIR + 'weather_train.csv')
weather_test = pd.read_csv(DATA_DIR + 'weather_test.csv')
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')

In [59]:
map_meter2desc = {0: 'electricity', 1: 'chilledwater', 2: 'steam', 3: 'hotwater'}

In [60]:
numerical = [
    'square_feet', 
    'year_built', 
    'floor_count', 
    'air_temperature', 
    'cloud_coverage', 
    'dew_temperature',
    'precip_depth_1_hr',
    'sea_level_pressure',
    'wind_speed',
]

categorical = [
    'primary_use',
]

# Combine table

In [61]:
df = train
df = df.merge(building_metadata, on='building_id')
df = df.merge(weather_train, on=['site_id', 'timestamp'])

In [62]:
df['log_meter_reading'] = df['meter_reading'].apply(lambda x: np.log(x + 1))
df['timestamp'] = pd.to_datetime(df['timestamp'])
# df = df.query('meter_reading > 0')

In [63]:
# df0 = df.query('meter == 0')
# df1 = df.query('meter == 1')
# df2 = df.query('meter == 2')
# df3 = df.query('meter == 3')
df = df.query('meter == {}'.format(METER)).reset_index(drop=True)

# Remove the problematic data

In [64]:
df = df.query('not (meter == 0 and site_id == 0 and timestamp < "2016-05-21")').reset_index(drop=True)

# Feature pre-processing

In [65]:
df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,log_meter_reading
0,106,3,2016-01-01,0.000,1,Education,5374,NaN,4.0,3.8,NaN,2.4,NaN,1020.9,240.0,3.1,0.000000
1,109,3,2016-01-01,0.000,1,Education,56995,1953.0,6.0,3.8,NaN,2.4,NaN,1020.9,240.0,3.1,0.000000
2,112,3,2016-01-01,96.978,1,Education,32206,NaN,6.0,3.8,NaN,2.4,NaN,1020.9,240.0,3.1,4.584743
3,113,3,2016-01-01,19.597,1,Education,100481,1958.0,9.0,3.8,NaN,2.4,NaN,1020.9,240.0,3.1,3.025145
4,114,3,2016-01-01,100.000,1,Education,139683,1958.0,13.0,3.8,NaN,2.4,NaN,1020.9,240.0,3.1,4.615121


In [66]:
# sns.boxplot(
#     y=df['floor_count'],#M orient='h',
# )

# sns.boxplot(
#     y=df.merge(
#         df.groupby('building_id').apply(lambda x: x[['timestamp', 'precip_depth_1_hr']].sort_values('timestamp').rolling('72h', on='timestamp').mean().set_index('timestamp')).reset_index(),
#         on=['building_id', 'timestamp'], how='left', suffixes=('', '_72h_mean'),
#     )['precip_depth_1_hr_72h_mean'].apply(lambda x: np.floor(x/10)*10),
#     x=df['log_meter_reading'],
#     orient='h',
# )

In [67]:
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.compose import make_column_transformer

feature_preproc = make_pipeline(
    FeatureUnion([
        ('numeric_features', make_pipeline(
            FunctionTransformer(lambda x: x[numerical], validate=False),
            FunctionTransformer(lambda x: x.assign(**{'precip_depth_1_hr': lambda y: y['precip_depth_1_hr'].clip(lower=0).fillna(0)}), validate=False),
            SimpleImputer(strategy="median"),
            #StandardScaler(),
        )),
        ('categorical_features', make_pipeline(
            FunctionTransformer(lambda x: x[categorical], validate=False),
            SimpleImputer(strategy="most_frequent"),
            OneHotEncoder(sparse=False),
        )),
        ('precip_depth_1_hr_isnan', make_pipeline(
            FunctionTransformer(lambda x: x[['precip_depth_1_hr']], validate=False),
            MissingIndicator(error_on_new=False, sparse=False),
        )),
        ('wind_direction', make_pipeline(
            FunctionTransformer(lambda x: pd.concat((
                    x['wind_direction'].apply(lambda x: np.sin(x/360 * 2*np.pi)),
                    x['wind_direction'].apply(lambda x: np.cos(x/360 * 2*np.pi)),
                ), axis='columns'), validate=False),
            SimpleImputer(strategy="median"),
        )),
        ('month', make_pipeline(
            FunctionTransformer(lambda x: pd.concat((
                    x['timestamp'].dt.month.apply(lambda x: np.sin(x/12 * 2*np.pi)),
                    x['timestamp'].dt.month.apply(lambda x: np.cos(x/12 * 2*np.pi)),
                ), axis='columns'), validate=False),
            SimpleImputer(strategy="median"),
        )), 
     ]),
)

feature_preproc.fit(df.sample(int(SUBSAMPLE_FEATURE_PREPROC * df.shape[0])));

feature_names = numerical \
    + [ re.sub(r"^(?:x)([0-9])", lambda m: categorical[int(m.group(1))], x) \
         for x in feature_preproc.steps[-1][-1].transformer_list[1][-1].steps[-1][-1].get_feature_names().tolist() ] \
    + ['precip_depth_1_hr_isnan'] \
    + ['wind_direction_sin', 'wind_direction_cos'] \
    + ['month_sin', 'month_cos']

In [68]:
np.random.seed(42)
idx_train = np.random.choice(df.shape[0], int(SUBSAMPLE_TRAIN * df.shape[0]), replace=False)

idx = np.zeros(df.shape[0]).astype(bool)
idx[idx_train] = True
idx_train = idx

In [69]:
X_train = pd.DataFrame(feature_preproc.transform(df.iloc[idx_train, :]), columns=feature_names)
y_train = df.iloc[idx_train]['log_meter_reading']

In [70]:
X_train.head()

,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_speed,primary_use_Education,...,primary_use_Healthcare,primary_use_Lodging/residential,primary_use_Office,primary_use_Public services,primary_use_Technology/science,precip_depth_1_hr_isnan,wind_direction_sin,wind_direction_cos,month_sin,month_cos
0,5374.0,1968.0,4.0,3.8,0.0,2.4,0.0,1020.9,3.1,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,-0.866025,-0.5,0.5,0.866025
1,56995.0,1953.0,6.0,3.8,0.0,2.4,0.0,1020.9,3.1,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,-0.866025,-0.5,0.5,0.866025
2,32206.0,1968.0,6.0,3.8,0.0,2.4,0.0,1020.9,3.1,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,-0.866025,-0.5,0.5,0.866025
3,100481.0,1958.0,9.0,3.8,0.0,2.4,0.0,1020.9,3.1,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,-0.866025,-0.5,0.5,0.866025
4,139683.0,1958.0,13.0,3.8,0.0,2.4,0.0,1020.9,3.1,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,-0.866025,-0.5,0.5,0.866025


# Train

In [71]:
from xgboost import XGBRegressor

model = XGBRegressor()

In [72]:
!date

Mon Nov  4 10:59:37 EST 2019


In [73]:
model.fit(X_train, y_train)

[10:59:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [74]:
!date

Mon Nov  4 11:00:51 EST 2019


In [75]:
y_pred = model.predict(X_train)

In [76]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_train, y_pred)

3.320781344815378

# Validate

In [77]:
X_val = pd.DataFrame(feature_preproc.transform(df.iloc[~idx_train, :]), columns=feature_names)
y_val = df.iloc[~idx_train]['log_meter_reading']

In [78]:
X_val.shape, idx_train.sum()

((126143, 22), 1135283)

In [79]:
y_pred = model.predict(X_val)

In [80]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_val, y_pred)

3.3332351891259457

# Feature importance

In [ ]:
pd.DataFrame({
    'feature_name': feature_names,
    'feature_importance': model.feature_importances_,
}).sort_values(by='feature_importance', ascending=False)

# Test

In [25]:
df = test
df = df.merge(building_metadata, on='building_id')
df = df.merge(weather_test, on=['site_id', 'timestamp'])

In [ ]:
df = df.query('meter == {}'.format(METER))

In [ ]:
X_test = pd.DataFrame(feature_preproc.transform(df[numerical + categorical]), columns=feature_names)

In [24]:
y_pred = model.predict(X_test)

In [25]:
submit = pd.DataFrame({'id': df['row_id'], 'meter_reading': np.round(np.exp(y_pred) - 1, 4)})

In [26]:
submit.to_csv('output/result_meter{}_test.csv'.format(METER), index=False)

# Save model

In [81]:
import dill as pickle

with open('model/model_xgboost_meter{}.p'.format(METER), 'wb') as file:
    pickle.dump(model, file)
    
with open('model/feature_preproc_meter{}.p'.format(METER), 'wb') as file:
    pickle.dump(feature_preproc, file)